In [0]:
!pip install highspy 

In [0]:
%restart_python

In [0]:
import highspy

from highspy import Highs
model = Highs()
print(f"HiGHS version from model: {model.version()}")

In [0]:
import numpy as np

# Define the supply chain edges
edges = [
    {"from": "Supplier", "to": "Factory1", "capacity": 100},
    {"from": "Supplier", "to": "Factory2", "capacity": 100},
    {"from": "Factory1", "to": "Warehouse", "capacity": 80},
    {"from": "Factory2", "to": "Warehouse", "capacity": 80},
]

nodes = ["Supplier", "Factory1", "Factory2", "Warehouse"]
disabled_node = "Factory2"  # simulate removing this node

# Filter out edges connected to the removed node
active_edges = [e for e in edges if disabled_node not in (e["from"], e["to"])]

print("Disabled node:", disabled_node)
print("Active edges after removal:")
for e in active_edges:
    print(f"{e['from']} → {e['to']}, capacity: {e['capacity']}")



In [0]:


# Flow balance (supply/demand) - adjusted for feasibility
balance = {node: 0 for node in nodes}
balance["Supplier"] = 80  
balance["Warehouse"] = -80

# Objective and variable bounds
num_vars = len(active_edges)
costs = np.array([1.0] * num_vars, dtype=np.double)
lower_bounds = np.zeros(num_vars, dtype=np.double)
upper_bounds = np.array([e["capacity"] for e in active_edges], dtype=np.double)

# Build constraint matrix for network flow
constraint_nodes = [n for n in nodes if n != disabled_node]
num_constraints = len(constraint_nodes)

# Initialize the model
model = Highs()

# Add variables with bounds only
for i in range(num_vars):
    model.addVar(lower_bounds[i], upper_bounds[i])
    
# Set the objective coefficients separately
for i in range(num_vars):
    model.changeColCost(i, costs[i])

# Build and add constraints
for i, node in enumerate(constraint_nodes):
    # Create a constraint for each node's flow balance
    constraint_coeffs = []
    constraint_indices = []
    
    for j, edge in enumerate(active_edges):
        if edge["from"] == node:
            constraint_coeffs.append(1.0)  # Outflow is positive
            constraint_indices.append(j)
        elif edge["to"] == node:
            constraint_coeffs.append(-1.0)  # Inflow is negative
            constraint_indices.append(j)
    
    # Add the constraint: flow balance equals node's supply/demand
    # Convert lists to numpy arrays with correct dtypes
    indices_array = np.array(constraint_indices, dtype=np.int32)
    coeffs_array = np.array(constraint_coeffs, dtype=np.double)
    model.addRow(float(balance[node]), float(balance[node]), len(indices_array), indices_array, coeffs_array)

# Run the optimization
model.run()

# Get solution status
status = model.getModelStatus()
print(f"Solution status: {status}")

# Output results
if status == 7:  # Optimal solution found
    col_values = []
    for i in range(num_vars):
        col_values.append(model.getSolution().col_value[i])
    
    print("\n✅ Optimization result:")
    print(f"Total flow: {sum(col_values)}")
    for val, e in zip(col_values, active_edges):
        print(f"{e['from']} → {e['to']}: {val} units")
else:
    print("No optimal solution found")